MNIST dataset with 99.40+ % validation Accuracy

In [1]:
%tensorflow_version 2.x
%load_ext tensorboard

TensorFlow 2.x selected.


In [2]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

#importing tensorflow
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten,BatchNormalization,MaxPooling2D
from tensorflow.keras.models import Model
import random as rn
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import mnist


Using TensorFlow backend.


In [0]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
X = X_train
y = y_train

test_x = X_test

In [0]:
X = X / 255.0
test_x = test_x / 255.0

In [0]:
X.shape

(60000, 28, 28)

In [0]:
# Reshape 
X = X.reshape(-1,28,28,1)
test_x = test_x.reshape(-1,28,28,1)

In [0]:
# Encode labels 
y = to_categorical(y)

print(f"Label size {y.shape}")

Label size (60000, 10)


In [0]:
# Split the train and the validation set for the fitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [0]:

batch_size = 64

In [0]:
# With data augmentation to prevent overfitting

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images



train_gen = datagen.flow(X_train, y_train, batch_size=batch_size)
test_gen = datagen.flow(X_test, y_test, batch_size=batch_size)

In [0]:
import datetime
!rm -rf ./logs/ 
#tf.reset_default_graph()
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [0]:
def scheduler(epoch):
  if epoch >= 40 and epoch <50:
    return 0.0001
  elif (epoch >= 60 and epoch <80):
    return 0.001
  
  elif (epoch >= 81 and epoch <99):
    return 0.0001

  else:
    return 0.001
  

In [0]:
filepath="weights.hdf5"
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.datasets import mnist

checkpoint_1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)


In [3]:
from keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
#model.add(BatchNormalization())   

model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform'))
#model.add(BatchNormalization())   

model.add(Conv2D(8, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
#model.add(BatchNormalization())   

model.add(Flatten())
model.add(Dense(13, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))
	
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        4624      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 8)           1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 13)                1

In [0]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=0.001),metrics=["accuracy"])

First 300 Epoch with 99.22 % validation accuracy

In [0]:
# Fit the model

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_graph=True,histogram_freq=1)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit_generator(train_gen, 
                              epochs = 300,
                              #use_multiprocessing=True, 
                              #workers=5,
                              steps_per_epoch = X.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = X_test.shape[0] // batch_size,
                              callbacks = [tensorboard_callback,checkpoint_1]
                              
                              )

Train for 937 steps, validate for 93 steps
Epoch 1/300
934/937 [============================>.] - ETA: 0s - loss: 0.0932 - accuracy: 0.9709
Epoch 00001: val_accuracy improved from 0.97379 to 0.97396, saving model to weights.hdf5
937/937 [==============================] - 17s 19ms/step - loss: 0.0933 - accuracy: 0.9708 - val_loss: 0.0871 - val_accuracy: 0.9740
Epoch 2/300
933/937 [============================>.] - ETA: 0s - loss: 0.0883 - accuracy: 0.9732
Epoch 00002: val_accuracy improved from 0.97396 to 0.97581, saving model to weights.hdf5
937/937 [==============================] - 17s 19ms/step - loss: 0.0885 - accuracy: 0.9732 - val_loss: 0.0814 - val_accuracy: 0.9758
Epoch 3/300
935/937 [============================>.] - ETA: 0s - loss: 0.0849 - accuracy: 0.9736
Epoch 00003: val_accuracy did not improve from 0.97581
937/937 [==============================] - 17s 18ms/step - loss: 0.0848 - accuracy: 0.9736 - val_loss: 0.0837 - val_accuracy: 0.9743
Epoch 4/300
934/937 [=============

Second 300 Epoch with 99.37 % validation accuracy (600 Epoch till)

In [0]:
# Fit the model

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_graph=True,histogram_freq=1)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit_generator(train_gen, 
                              epochs = 300,
                              #use_multiprocessing=True, 
                              #workers=5,
                              steps_per_epoch = X.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = X_test.shape[0] // batch_size,
                              callbacks = [tensorboard_callback,checkpoint_1]
                              
                              )

Train for 937 steps, validate for 93 steps
Epoch 1/300
934/937 [============================>.] - ETA: 0s - loss: 0.0383 - accuracy: 0.9884
Epoch 00001: val_accuracy did not improve from 0.99227
937/937 [==============================] - 18s 19ms/step - loss: 0.0382 - accuracy: 0.9884 - val_loss: 0.0346 - val_accuracy: 0.9896
Epoch 2/300
935/937 [============================>.] - ETA: 0s - loss: 0.0379 - accuracy: 0.9884
Epoch 00002: val_accuracy did not improve from 0.99227
937/937 [==============================] - 18s 19ms/step - loss: 0.0378 - accuracy: 0.9884 - val_loss: 0.0337 - val_accuracy: 0.9892
Epoch 3/300
934/937 [============================>.] - ETA: 0s - loss: 0.0358 - accuracy: 0.9886
Epoch 00003: val_accuracy did not improve from 0.99227
937/937 [==============================] - 18s 19ms/step - loss: 0.0358 - accuracy: 0.9886 - val_loss: 0.0301 - val_accuracy: 0.9908
Epoch 4/300
934/937 [============================>.] - ETA: 0s - loss: 0.0367 - accuracy: 0.9883
Epoch

Third 300 Epoch with 99.37 % validation accuracy (900 Epoch till)

In [0]:
# Fit the model

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_graph=True,histogram_freq=1)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit_generator(train_gen, 
                              epochs = 300,
                              #use_multiprocessing=True, 
                              #workers=5,
                              steps_per_epoch = X.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = X_test.shape[0] // batch_size,
                              callbacks = [tensorboard_callback,checkpoint_1]
                              
                              )

Train for 937 steps, validate for 93 steps
Epoch 1/300
934/937 [============================>.] - ETA: 0s - loss: 0.0341 - accuracy: 0.9894
Epoch 00001: val_accuracy did not improve from 0.99378
937/937 [==============================] - 19s 21ms/step - loss: 0.0341 - accuracy: 0.9894 - val_loss: 0.0330 - val_accuracy: 0.9909
Epoch 2/300
936/937 [============================>.] - ETA: 0s - loss: 0.0340 - accuracy: 0.9894
Epoch 00002: val_accuracy did not improve from 0.99378
937/937 [==============================] - 19s 21ms/step - loss: 0.0339 - accuracy: 0.9894 - val_loss: 0.0283 - val_accuracy: 0.9911
Epoch 3/300
936/937 [============================>.] - ETA: 0s - loss: 0.0327 - accuracy: 0.9896
Epoch 00003: val_accuracy did not improve from 0.99378
937/937 [==============================] - 19s 20ms/step - loss: 0.0327 - accuracy: 0.9896 - val_loss: 0.0282 - val_accuracy: 0.9916
Epoch 4/300
934/937 [============================>.] - ETA: 0s - loss: 0.0343 - accuracy: 0.9900
Epoch

Forth 300 Epoch with 99.42 % validation accuracy (1200 Epoch till)

I get this Warning/Error....
Buffered data was truncated after reaching the output size limit......
but got desired results...hence fine....

In [0]:
# Fit the model

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,write_graph=True,histogram_freq=1)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit_generator(train_gen, 
                              epochs = 300,
                              #use_multiprocessing=True, 
                              #workers=5,
                              steps_per_epoch = X.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = X_test.shape[0] // batch_size,
                              callbacks = [tensorboard_callback,checkpoint_1]
                              
                              )

Train for 937 steps, validate for 93 steps
Epoch 1/300
936/937 [============================>.] - ETA: 0s - loss: 0.0318 - accuracy: 0.9900
Epoch 00001: val_accuracy did not improve from 0.99378
937/937 [==============================] - 18s 19ms/step - loss: 0.0319 - accuracy: 0.9900 - val_loss: 0.0259 - val_accuracy: 0.9914
Epoch 2/300
934/937 [============================>.] - ETA: 0s - loss: 0.0328 - accuracy: 0.9900
Epoch 00002: val_accuracy did not improve from 0.99378
937/937 [==============================] - 18s 19ms/step - loss: 0.0327 - accuracy: 0.9900 - val_loss: 0.0336 - val_accuracy: 0.9889
Epoch 3/300
935/937 [============================>.] - ETA: 0s - loss: 0.0320 - accuracy: 0.9902
Epoch 00003: val_accuracy did not improve from 0.99378
937/937 [==============================] - 17s 18ms/step - loss: 0.0319 - accuracy: 0.9902 - val_loss: 0.0272 - val_accuracy: 0.9904
Epoch 4/300
934/937 [============================>.] - ETA: 0s - loss: 0.0313 - accuracy: 0.9904
Epoch